In [2]:
%store -r setup_dependencies_passed

In [3]:
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")

In [4]:
print(setup_dependencies_passed)

True


In [5]:
%store -r setup_s3_bucket_passed

In [6]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")

In [7]:
print(setup_s3_bucket_passed)

True


In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [29]:
s3_private_path_csv = "s3://{}/dataset/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-652903355321/dataset/csv


In [30]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Specify the relative path to the data file within the current directory
file_path = os.path.join(current_directory, 'dataset.csv')

print("Path of the data file:", file_path)

Path of the data file: /root/aai-540-labs/MLOpsFinalProject/dataset.csv


In [31]:
s3 = boto3.resource('s3')
s3.Object(bucket, 'dataset/csv/dataset.csv').put(Body=open('/root/aai-540-labs/MLOpsFinalProject/dataset.csv', 'rb'))

{'ResponseMetadata': {'RequestId': 'AH3046JQJSC88YAJ',
  'HostId': 'xo5z7Udwo25snRY5NbSTWbT8K3qXANXhuV5+fHzUalNqT9JB1DvFVUkFuowKTgazbCuQ/iKeXj4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xo5z7Udwo25snRY5NbSTWbT8K3qXANXhuV5+fHzUalNqT9JB1DvFVUkFuowKTgazbCuQ/iKeXj4=',
   'x-amz-request-id': 'AH3046JQJSC88YAJ',
   'date': 'Sat, 11 May 2024 15:54:53 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e50a70711e823b75c8ebe231468f5faf"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e50a70711e823b75c8ebe231468f5faf"',
 'ServerSideEncryption': 'AES256'}

In [32]:
print(s3_private_path_csv)

s3://sagemaker-us-east-1-652903355321/dataset/csv


In [33]:
!aws s3 ls $s3_private_path_csv/

2024-05-11 15:54:53   20118244 dataset.csv


In [34]:
!aws s3 ls s3://sagemaker-us-east-1-652903355321

                           PRE amazon-reviews-pds/
                           PRE athena/
                           PRE dataset/
2024-05-11 15:42:49   20118244 dataset.csv


In [35]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/amazon-reviews-pds/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

/tmp/ipykernel_21/4286372116.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [37]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [39]:
ingest_create_athena_db_passed = False

In [41]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

In [42]:
from pyathena import connect

In [43]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [44]:
print(s3_staging_dir)

s3://sagemaker-us-east-1-652903355321/athena/staging


In [64]:
database_name = "assgnaws"
table_name = "HomeworkDataset"
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS assgnaws


In [65]:
import pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_21/1322682284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [66]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_21/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,assgnaws
1,default
2,dsoaws


In [67]:
# Set Athena parameters
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [68]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         srnbr int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit string,
         danceability string,
         energy string,
         key int,
         loudness string,
         mode int,
         speechiness string,
         acousticness string,
         instrumentalness string,
         liveness string,
         valence string,
         tempo string,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path_csv
)

print(statement)
import pandas as pd

pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS assgnaws.HomeworkDataset(
         srnbr int,
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit string,
         danceability string,
         energy string,
         key int,
         loudness string,
         mode int,
         speechiness string,
         acousticness string,
         instrumentalness string,
         liveness string,
         valence string,
         tempo string,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-652903355321/dataset/csv'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


/tmp/ipykernel_21/3618033062.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [69]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_21/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,dataset
1,homeworkdataset


In [70]:
track_genre = "acoustic"

statement = """SELECT * FROM {}.{}
    WHERE track_genre = '{}' LIMIT 100""".format(
    database_name, table_name, track_genre
)

print(statement)

SELECT * FROM assgnaws.HomeworkDataset
    WHERE track_genre = 'acoustic' LIMIT 100


In [71]:
df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_21/2446512133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,srnbr,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,1.01e-06,0.358,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.42,0.166,...,-17.235,1,0.0763,0.924,5.56e-06,0.101,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.359,...,-9.734,1,0.0557,0.21,0.0,0.117,0.12,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.905,7.07e-05,0.132,0.143,181.74,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.443,...,-9.681,1,0.0526,0.469,0.0,0.0829,0.167,119.949,4,acoustic
